# Исследование опроса клиентов телекомунникацонной компании

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

Чтобы оценить результаты опроса, оценки обычно делят на три группы:

* 9-10 баллов — «cторонники» (англ. promoters);
* 7-8 баллов — «нейтралы» (англ. passives);
* 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике. 

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса. 

In [68]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [69]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [70]:
query = """
SELECT 
    user_id, lt_day, age, gender_segment, os_name, cpe_type_name, 
    location.country, location.city,
    ags.title AS age_segment,
    lts.title AS lifetime_segment,
    ts.title AS traffic_segment,
    nps_score,
    CASE
        WHEN nps_score >= 9 THEN "promoters"
        WHEN nps_score >= 7 THEN "passives"
        ELSE "detractors"
    END AS nps_group,
    CASE
        WHEN lts.title < "06 13-24" THEN True
        ELSE False
    END AS is_new
FROM user LEFT JOIN location ON user.location_id = location.location_id
LEFT JOIN lifetime_segment AS lts ON user.lt_gr_id = lts.lt_gr_id
LEFT JOIN traffic_segment AS ts ON user.tr_gr_id = ts.tr_gr_id
LEFT JOIN age_segment AS ags ON user.age_gr_id = ags.age_gr_id
"""

In [71]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,lifetime_segment,traffic_segment,nps_score,nps_group,is_new
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,08 36+,04 1-5,10,promoters,0
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,08 36+,04 1-5,10,promoters,0
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,06 13-24,08 20-25,10,promoters,0


Сохарняем результаты, выгружаем их и загружаем в Tableau.

In [91]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

[Ссылка на проект](https://public.tableau.com/views/SP_1_17155236052790/sheet14?:language=en-US&publish=yes&:sid=&:display_count=n&:origin=viz_share_link)